#More Experiments with the Dataset from Raspberry Turk Project.

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

I am assume that the pieces will be black and gold, however if they are differing colors the code can be updated.

Now in the original, the author included these functions:
*   from raspberryturk.core.data.dataset import Dataset
*   d = Dataset.load_file(project.path('data', 'processed', 'chess_piece_presence.npz'))

Now since the goal is to make a general version for our purpose, I'm not sure how to handle this; I assume we may eventually create our own dataset for testing purposes but for now I just left them as is.

In [ ]:
def convert_imgs(X):
    norm_imgs = X.reshape((-1,60,60,3))
    bgr_imgs = ((norm_imgs + 1.0) * 127.5).astype(np.uint8)
    return np.array([cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB) for bgr_img in bgr_imgs])

imgs = convert_imgs(d.X_train)
validation_imgs = convert_imgs(d.X_val)
label_names = ["Empty", "Black", "Gold"]

In [ ]:
def plot_image(img_index, validation=False, title=None):
    img = (validation_imgs if validation else imgs)[img_index]
    if title is None:
        label = (d.y_val if validation else d.y_train)[img_index]
        title = label_names[label]
    plt.title(title)
    plt.imshow(img)
    plt.show()

In [ ]:
x_kern = np.arange(0, 4, 1, float)
y_kern = x_kern[:,np.newaxis]
x_kern0 = y_kern0 = 4 // 2
OPENING_KERNEL = np.uint8(np.exp(-4*np.log(2) * ((x_kern-x_kern0)**2 + (y_kern-y_kern0)**2) / 2**2) * 255)

def mask(img, lower, upper):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, OPENING_KERNEL)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, OPENING_KERNEL)
    return closing

We may want to consider making a function to convert RBG values to HSV as I've been seeing differing values for the HSV of gold and creating the function may allow us to have the most accurate range. There's a few I've seen so I'll review and see which is better for our case.

In [ ]:
# HSV Color Ranges
BLACK_LOWER = (0,0,0)
BLACK_UPPER = (180,255,30)
GOLD_LOWER = (238, 188, 29)
GOLD_UPPER = (241, 229, 172)

def black_mask(img):
    return mask(img, BLACK_LOWER, BLACK_UPPER)

def gold_mask(img):
    return mask(img, GOLD_LOWER, GOLD_UPPER)

def combined_mask(img):
    return black_mask(img) + gold_mask(img)

In [ ]:
c = np.zeros((400, 400, 3), dtype=np.uint8)
start_index = 1000
for i in range(start_index, start_index + 200):
    img = imgs[i]
    mask_img = cv2.bitwise_or(black_mask(img), gold_mask(img))
    x = (i % 10) * 40
    y = ((i % 200) / 10) * 20
    c[y:y+20,x:x+20,:] = cv2.resize(img, (20,20))
    c[y:y+20,x+20:x+40,0] = cv2.resize(mask_img, (20,20))
plt.figure(figsize=(10,10))
plt.axis('off')
plt.imshow(c)
plt.show()

In [ ]:
def predict(img):
    threshold = 5.0
    if np.sum(black_mask(img)) > threshold:
        return 1
    elif np.sum(gold_mask(img)) > threshold:
        return 2
    else:
        return 0

In [ ]:
image_index = 405
actual_label = label_names[d.y_val[image_index]]
predicted_label = label_names[predict(validation_imgs[image_index])]
title = "Actual: {} Predicted: {}".format(actual_label, predicted_label)
plot_image(image_index, validation=True, title=title)

In [ ]:
size = imgs.shape[1]
x = np.arange(0, size, 1, float)
y = x[:,np.newaxis]
x0 = y0 = size // 2
MASK_WEIGHTS = np.exp(-4*np.log(2) * ((x-x0)**2 + (y-y0)**2) / (size*0.15)**2)

def weighted_gold_mask(img):
    return gold_mask(img) * MASK_WEIGHTS

def weighted_black_mask(img):
    return black_mask(img) * MASK_WEIGHTS

def weighted_predict(img):
    threshold = 5.0
    if np.sum(weighted_black_mask(img)) > threshold:
        return 1
    elif np.sum(weighted_gold_mask(img)) > threshold:
        return 2
    else:
        return 0